In [3]:
## Instalando dependencias 
! pip install --no-cache-dir -r requirements.txt

In [1]:
## Importando os class
from functions.trino_provider import TrinoProvider
from functions.s3_functions import s3Function
from functions.format_data import FormatData
import pandas as pd
import csv

In [2]:
## Inicializando 
trinoProvider = TrinoProvider()
s3 = s3Function()
fmt = FormatData()

In [3]:
## Declarando variaveis
BUCKET_NAME='apresentacao'
CATALOG='minio'
SCHEMA='dev'

In [4]:
# MINIO

## Lista  de buckets
s3.get_buckets()

['apresentacao']

In [41]:
## Criando um bucket
s3.create_bucket(BUCKET_NAME)

Creating new bucket: apresentacao2


'Bucket: apresentacao2 created!'

In [5]:
## Lendo arquivo local e inserindo no bucket

orders = pd.read_csv('../files/orders.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/orders/orders', data=orders, file_format='csv')

ingredients = pd.read_csv('../files/ingredients.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/ingredients/ingredients', data=ingredients, file_format='csv')

items = pd.read_csv('../files/items.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/items/items', data=items, file_format='csv')

recipe = pd.read_csv('../files/recipe.csv',  sep=',', quoting=csv.QUOTE_ALL, skipinitialspace = True, encoding='ISO-8859-1')
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/recipe/recipe', data=recipe, file_format='csv')

{'data': b'row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out\n1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out\n2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in\n3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out\n4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out\n5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out\n6,ORD006,2024-02-12 07:28:20,It001,1,Morgan,in\n7,ORD006,2024-02-12 07:28:20,It016,1,Morgan,in\n8,ORD007,2024-02-12 07:33:58,It005,1,Riley,out\n9,ORD007,2024-02-12 07:33:58,It020,1,Riley,\n10,ORD008,2024-02-12 07:39:02,It006,1,Cameron,in\n11,ORD008,2024-02-12 07:39:01,It018,1,Cameron,\n12,ORD009,2024-02-12 07:44:02,It023,1,Quinn,out\n13,ORD009,2024-02-12 07:44:03,It011,1,Quinn,\n14,ORD010,2024-02-12 07:49:04,It024,1,Peyton,out\n15,ORD010,2024-02-12 07:49:05,It014,1,Peyton,\n16,ORD011,2024-02-12 07:53:06,It003,1,Brooke,out\n17,ORD012,2024-02-12 07:58:07,It007,1,Blake,out\n18,ORD013,2024-02-12 08:03:08,It009,1,Charlie,in\n19,ORD013,2024-02-12 08:03:09,It021,1,Charlie,\n20,ORD01

In [6]:
## Lista de objetos no bucket
s3.list_objects(BUCKET_NAME, dir=SCHEMA)

['dev/bronze/recipe_ingredients',
 'dev/ingredients/ingredients',
 'dev/items/items',
 'dev/orders/orders',
 'dev/recipe/recipe']

In [7]:
recipe.head()

,row_id,recipe_id,ing_id,quantity
0,1,HDR-CAP-MD,ING001,8
1,2,HDR-CAP-MD,ING002,130
2,3,HDR-CAP-LG,ING001,10
3,4,HDR-CAP-LG,ING002,180
4,5,HDR-LAT-MD,ING001,8


In [ ]:
## Retorno em df
## Exemplo de como ler arquivo direto do bucket(MinIo) 
orders = s3.load_file_s3(path='dev/orders/orders', bucket_name=BUCKET_NAME, format='csv')
items = s3.load_file_s3(path='dev/items/items', bucket_name=BUCKET_NAME, format='csv')
ingredients = s3.load_file_s3(path='dev/ingredients/ingredients', bucket_name=BUCKET_NAME, format='csv')
recipe = s3.load_file_s3(path='dev/ingredients/ingredients', bucket_name=BUCKET_NAME, format='csv')

In [50]:
# TRINO
trinoProvider.test_connection()

[['trino', 'http://192.168.176.4:8080', '351', True, 'active']]

In [33]:
## Exemplos de chamadas 

## Ver catalago de dados
cursor = trinoProvider.conn()
cursor.execute('SHOW CATALOGS')
print(cursor.fetchall())

trinoProvider.get_result_pandas('SHOW CATALOGS')


[['memory'], ['minio'], ['system'], ['tpcds'], ['tpch']]

Execute SQL: SHOW CATALOGS


,Catalog
0,memory
1,minio
2,system
3,tpcds
4,tpch


In [34]:
## Ver schemas no Trino
trinoProvider.get_result_pandas("SHOW SCHEMAS FROM minio LIKE 'dev'")


Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'


,Schema
0,dev


In [20]:
## lista de tabelas
trinoProvider.get_result_pandas("SHOW tables FROM minio.dev")


Execute SQL: SHOW tables FROM minio.dev


,Table
0,cancelled_diverted
1,ingredients
2,items
3,orders
4,recipe
5,us_flights
6,weather_meteo_by_airport


In [74]:
## Deletando uma tabela
trinoProvider.get_result_pandas("DROP TABLE minio.dev.items")


Execute SQL: DROP TABLE minio.dev.items


,result
0,True


In [ ]:
## Criando tabela no trino
key_s3= f"{SCHEMA}/orders"
trinoProvider.create_table_and_metadata(df=orders, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA,  key_s3=key_s3, table='orders', separator=',', format_file='csv')


In [ ]:
## Criando tabela no trino
key_s3= f"{SCHEMA}/items"
trinoProvider.create_table_and_metadata(df=items, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='items', separator=',', format_file='csv')

In [ ]:
## Criando tabela no trino
key_s3 = f"{SCHEMA}/ingredients"
trinoProvider.create_table_and_metadata(df=ingredients, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='ingredients', separator=',', format_file='csv')

In [11]:
## Criando tabela  no trino
key_s3 = f"{SCHEMA}/recipe"
trinoProvider.create_table_and_metadata(df=recipe, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA, key_s3=key_s3, table='recipe', separator=',', format_file='csv')

NAME:  row_id  TYPE:  int64
NAME:  recipe_id  TYPE:  object
NAME:  ing_id  TYPE:  object
NAME:  quantity  TYPE:  int64

Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'
Schema dev exist

Execute SQL: SHOW TABLES FROM minio.dev LIKE 'recipe'
Table minio.dev.recipe exist


,0
0,recipe


In [12]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.orders").head()


Execute SQL: SELECT * FROM minio.dev.orders


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out


In [20]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.items").head()


Execute SQL: SELECT * FROM minio.dev.items


,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,"""Hot Drinks""",Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,"""Hot Drinks""",Large,3.75
2,It003,HDR-LAT-MD,Latte,"""Hot Drinks""",Medium,3.45
3,It004,HDR-LAT-LG,Latte,"""Hot Drinks""",Large,3.75
4,It005,HDR-FLT,"""Flat White""","""Hot Drinks""",N/A,3.15


In [82]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.ingredients").head()


Execute SQL: SELECT * FROM minio.dev.ingredients


,ing_id,ing_name,ing_weight,ing_meas,ing_price
0,ING001,Espresso beans,1000,grams,12.00
1,ING002,Whole Milk,1000,ml,1.20
2,ING003,Cheddar,500,grams,7.45
3,ING004,Mozzarella,500,grams,5.00
4,ING005,Whipped cream,300,ml,1.35


In [83]:
sql = """
    SELECT o.* 
         , i.*
      FROM minio.dev.orders as o
    INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id
"""
trinoProvider.get_result_pandas(sql).head()


Execute SQL: 
    SELECT o.* 
         , i.*
      FROM minio.dev.orders as o
    INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id



,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out,item_id,sku,item_name,item_cat,item_size,item_price
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out,It008,HDR-ESP,Espresso,Hot Drinks,,2.15
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in,It014,HDR-HCH-LG,Hot Chocolate,Hot Drinks,Large,4.60
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out,It008,HDR-ESP,Espresso,Hot Drinks,,2.15
3,4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out,It019,CDR-ICT-MD,Iced Tea,Cold Drinks,Medium,3.25
4,5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out,It024,SNK-SSM,Sandwich Salami&Mozzarella,Snacks,,5.50


In [87]:
sql = """
    SELECT o.order_id
         , o.cust_name
        , i.item_name
        , i.item_size
        , i.item_price
      FROM minio.dev.orders as o
     INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id

"""
trinoProvider.get_result_pandas(sql).head()


Execute SQL: 
    SELECT o.order_id
         , o.cust_name
        , i.item_name
        , i.item_size
        , i.item_price
      FROM minio.dev.orders as o
     INNER JOIN minio.dev.items as i
       ON o.item_id = i.item_id




,order_id,cust_name,item_name,item_size,item_price
0,ORD001,Alex,Espresso,,2.15
1,ORD002,Jordan,Hot Chocolate,Large,4.60
2,ORD003,Taylor,Espresso,,2.15
3,ORD004,Casey,Iced Tea,Medium,3.25
4,ORD005,Jamie,Sandwich Salami&Mozzarella,,5.50


In [13]:
sql = """
    SELECT ing.ing_id
         , ing.ing_name
         , ing.ing_price
         , r.recipe_id
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id
"""
recipe_ingredients = trinoProvider.get_result_pandas(sql)


Execute SQL: 
    SELECT ing.ing_id
         , ing.ing_name
         , ing.ing_price
         , r.recipe_id
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id



In [14]:
recipe_ingredients.head()

,ing_id,ing_name,ing_price,recipe_id
0,ING001,Espresso beans,12.0,CDR-CMO-LG
1,ING001,Espresso beans,12.0,CDR-CMO-MD
2,ING001,Espresso beans,12.0,CDR-CCF-LG
3,ING001,Espresso beans,12.0,CDR-CCF-MD
4,ING001,Espresso beans,12.0,HDR-WMO-LG


In [36]:
## Inserindo modificacao
s3.put_bucket(bucket_name=BUCKET_NAME, key='dev/bronze/recipe_ingredients/recipe_ingredients', data=recipe_ingredients, file_format='csv')

{'data': b'ing_id,ing_name,ing_price,recipe_id\nING001,Espresso beans,12.0,CDR-CMO-LG\nING001,Espresso beans,12.0,CDR-CMO-MD\nING001,Espresso beans,12.0,CDR-CCF-LG\nING001,Espresso beans,12.0,CDR-CCF-MD\nING001,Espresso beans,12.0,HDR-WMO-LG\nING001,Espresso beans,12.0,HDR-WMO-MD\nING001,Espresso beans,12.0,HDR-MOC-LG\nING001,Espresso beans,12.0,HDR-MOC-MD\nING001,Espresso beans,12.0,HDR-ESP\nING001,Espresso beans,12.0,HDR-CRM-LG\nING001,Espresso beans,12.0,HDR-CRM-MD\nING001,Espresso beans,12.0,HDR-FLT\nING001,Espresso beans,12.0,HDR-LAT-LG\nING001,Espresso beans,12.0,HDR-LAT-MD\nING001,Espresso beans,12.0,HDR-CAP-LG\nING001,Espresso beans,12.0,HDR-CAP-MD\nING002,Whole Milk,1.2,CDR-CMO-LG\nING002,Whole Milk,1.2,CDR-CMO-MD\nING002,Whole Milk,1.2,CDR-CCF-LG\nING002,Whole Milk,1.2,CDR-CCF-MD\nING002,Whole Milk,1.2,HDR-HCH-LG\nING002,Whole Milk,1.2,HDR-HCH-MD\nING002,Whole Milk,1.2,HDR-WMO-LG\nING002,Whole Milk,1.2,HDR-WMO-MD\nING002,Whole Milk,1.2,HDR-MOC-LG\nING002,Whole Milk,1.2,HDR-MO

In [34]:
## Criando tabela da transfomacao no trino
key_s3=f"{SCHEMA}/bronze/recipe_ingrediente"
trinoProvider.create_table_and_metadata(df=recipe_ingredients, bucket_name=BUCKET_NAME, catalog=CATALOG, schema=SCHEMA,  key_s3=key_s3, table='recipe_ingredients', separator=',', format_file='csv')

NAME:  ing_id  TYPE:  object
NAME:  ing_name  TYPE:  object
NAME:  ing_price  TYPE:  float64
NAME:  recipe_id  TYPE:  object

Execute SQL: SHOW SCHEMAS FROM minio LIKE 'dev'
Schema dev exist

Execute SQL: SHOW TABLES FROM minio.dev LIKE 'recipe_ingredients'
Creating table minio.dev.recipe_ingredients ...

Execute SQL: CREATE TABLE IF NOT EXISTS minio.dev.recipe_ingredients (
                        ing_id  varchar, 
ing_name  varchar, 
ing_price  double, 
recipe_id  varchar
                    )
                    WITH (
                        format = 'TEXTFILE',
                        skip_header_line_count = 1,
                        textfile_field_separator = ',',
                        external_location = 's3a://apresentacao//dev/bronze/recipe_ingredients.csv'
                    )
                    
Table minio.dev.recipe_ingredients criada, file format csv 


""


In [32]:
trinoProvider.get_result_pandas("DROP TABLE minio.dev.recipe_ingredients")


Execute SQL: DROP TABLE minio.dev.recipe_ingredients


('TrinoProvider.get_result_pandas -',
 TrinoUserError(type=USER_ERROR, name=TABLE_NOT_FOUND, message="line 1:1: Table 'minio.dev.recipe_ingredients' does not exist", query_id=20240429_224052_00143_v3tyh))

In [35]:
trinoProvider.get_result_pandas("SELECT * FROM minio.dev.recipe_ingredients")


Execute SQL: SELECT * FROM minio.dev.recipe_ingredients
Result empty


,ing_id,ing_name,ing_price,recipe_id


In [100]:
items.head()

,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75
4,It005,HDR-FLT,Flat White,Hot Drinks,NaN,3.15


In [ ]:
sql = """
    SELECT 
      FROM minio.dev.ingredients as ing
     INNER JOIN minio.dev.recipe as r
        ON ing.ing_id = r.ing_id
"""
ing_item001 = trinoProvider.get_result_pandas(sql)